<div style="
  padding: 5pt;
  border-style: solid;
  border-width: 1px;
  border-color: gray;
  border-radius: 10px;">
  
# **Python et intelligence artificielle**

# *Séance n°9 : Apprentissage supervisé et classification*

</div>

Dans cette séance, vous allez approfondir vos connaissances en **apprentissage supervisé** avec *Scikit-learn* en explorant le **concept de classification** appliqué à la détection d'anomalies. Vous apprendrez à évaluer vos modèles de manière robuste en utilisant d'autres **métriques d'évaluation** ainsi que la technique de **validation croisée**.

Vous apprendrez entre autres à :

- Analyser et visualiser un jeu de données afin d'en extraire des tendances.
- Entraîner un modèle de **classification binaire** et ajuster ses hyperparamètres.
- Appliquer plusieurs **métriques d'évaluation** pour déterminer la précision du modèle.
- Comparer différents modèles de classification (**régression logistique**, **k-NN** et **machine à vecteurs de support**).
- Utiliser la **validation croisée** pour évaluer la robustesse d'un modèle.

---

## Introduction

La **classification** est une méthode d'apprentissage où l'objectif est
de prédire une **classe** (catégorie) à partir des données d'entrée. Dans cette séance,
nous étudierons trois algorithmes pour la classification binaire, afin de déterminer
si un objet est une mine sous-marine ou une roche. Ce type de classification est
largement utilisée dans l'industrie afin d'anticiper les défaillances d'équipements,
mais aussi dans le domaine de la santé pour la détection de maladies ou encore dans
le domaine de la finance pour détecter les fraudes.

Les trois modèles que nous allons mettre en oeuvre sont :

- la **régression logistique**, qui permet de prédire une probabilité d'appartenance
  à une classe et est adaptée aux classes linéairement séparables ;
- le **k-plus proches voisins** (k-NN), qui classe une observation en fonction des
  $k$ voisins les plus proches dans l'espace des caractéristiques et qui convient aux
  relations plus complexes ;
- la **machine à vecteurs de support** (SVM) particulièrement efficace pour les
  problèmes de classification non linéaire.

### Description des données

Le jeu de données "`sonar.csv`" du sous-dossier "ressources" contient des mesures de
signaux sonar réfléchis par des objets sous-marins cylindriques et classe les signaux
en deux catégories : mines et roches.
La structure et la densité de l'objet (mine ou roche) influencent la façon dont les
ondes émise par le sonar sont réfléchies dans chaque bande de fréquence. Par exemple,
une mine, souvent en métal, pourrait produire des réflexions différentes par rapport
à une roche de même taille.

#### Structure et caractéristiques des données

Le fichier est décomposé en 208 lignes de 61 colonnes.

- Chacune des 60 premières colonnes représente l'intensité de réflexion du signal
  sonar dans un intervalle de fréquence donné. Ces valeurs correspondent aux amplitudes
  mesurées dans différentes bandes de fréquence, de la plus basse (caractéristique 1) à la
  plus élevée (caractéristique 60). Les amplitudes sont normalisées dans une plage entre
  0.0 et 1.0. Une valeur proche de 1 indique une forte intensité de réflexion dans cette
  bande de fréquence, tandis qu'une valeur proche de 0 indique une faible intensité.
- La dernière colonne représente la classe cible qui étiquette chaque échantillon comme
  "R" pour roche ou "M" pour mine.

Les 60 caractéristiques sont souvent corrélées entre elles, et une analyse préliminaire
(par exemple avec une carte de corrélation) peut vous permettre d'identifier les relations
entre fréquences et d'évaluer si certaines bandes de fréquences sont plus discriminantes
pour distinguer les deux classes.

### Régression logistique

La **régression logistique** est un modèle probabiliste conçu pour la **classification binaire**. Contrairement à la régression linéaire, elle utilise une fonction logistique pour transformer la sortie en une probabilité d'appartenance à une classe. Elle est bien adaptée pour des données linéairement séparables.

La probabilité qu'une observation appartienne à la classe positive est donnée par la **fonction sigmoïde** :

$$
P(y=1|X) = \sigma(z) = \frac{1}{1 + e^{-z}}
$$

où $z$ est une combinaison linéaire des variables explicatives :

$$
z = a_0 + a_1 X_1 + a_2 X_2 + \dots + a_n X_n
$$
et $\sigma(z)$ renvoie une valeur comprise entre 0 et 1 qui représente la probabilité que $y$ soit
égal à 1 (appartenance à la classe positive).

#### Décision et seuil

La régression logistique utilise un **seuil** généralement fixé à 0,5 pour la
prise de décision :

- $\text{Si } P(y=1|X) > \text{seuil} \Rightarrow y = 1 \quad \text{(classe positive)}$
- $\text{Si } P(y=1|X) \leq \text{seuil} \Rightarrow y = 0 \quad \text{(classe négative)}$

#### Fonction de coût et optimisation

La **log-vraisemblance** représente la probabilité d'observer les données étant
donné les paramètres du modèle. La **log-vraisemblance négative** est utilisée
comme **fonction de coût**, et elle est minimisée afin d'obtenir le meilleur
ajustement possible :

$$
J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} \left( y^{(i)} \log(h_{\theta}(x^{(i)})) + (1 - y^{(i)}) \log(1 - h_{\theta}(x^{(i)})) \right)
$$

où :

- $m$ est le nombre d'exemples dans le jeu de données d'entraînement ;
- $x^{(i)}$ est le $i$-ème exemple d'entraînement ;
- $y^{(i)}$ est l'étiquette de l'exemple $i$, soit 0 ou 1 ;
- $h_{\theta}(x^{(i)})$ est la probabilité prédite pour l'exemple $i$ donnée par la fonction sigmoïde.

L'objectif est de maximiser la vraisemblance (ou de minimiser sa version négative).

La classe `LogisticRegression` de *Scikit-learn* utilise des méthodes d'optimisation
comme la **descente de gradient** pour trouver les paramètres optimaux.

#### Exemple

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
from sklearn.linear_model import LogisticRegression

# Modèle de régression logistique
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)     # Entraînement
y_pred = log_reg.predict(X_test)  # Prédiction
```

</div>

### k-plus proche voisins

L'algorithme des **k-plus proches voisins** (ou k-NN) est un modèle de classification non
paramétrique :

1. Pour classer une observation, il identifie ses $k$ plus proches voisins en termes de distance.
2. La classe majoritaire parmi les $k$ voisins est assignée à l'observation.

La **distance euclidienne** est souvent utilisée pour mesurer la proximité des voisins.
L'algorithme k-NN est simple et fonctionne bien sur des petits jeux de données ou pour des
relations locales.

*Scikit-learn* fournit l'implémentation d'un k-NN avec la classe `KNeighborsClassifier`.

#### Exemple

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
from sklearn.neighbors import KNeighborsClassifier

# Instanciation du modèle k-NN avec k=3
knn = KNeighborsClassifier(n_neighbors=3)

# Entraînement du modèle
knn.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred_knn = knn.predict(X_test)
```

</div>

Remarque : k-NN permet aussi la régression.

### Machine à vecteurs de support

Une **machine à vecteurs de support** (SVM) est un modèle de classification (linéaire ou non linéaire) qui cherche à séparer des classes en maximisant la marge entre elles. La marge est définie comme la distance entre la frontière de décision et les points les plus proches de chaque classe, appelés *vecteurs de support*.

- Si les données sont linéairement séparables, la SVM trouve l'hyperplan (une ligne en 2D, un plan en 3D, etc.) qui maximise la marge.
- Pour des données non linéairement séparables, la SVM utilise des **noyaux** pour transformer les données dans un espace de dimension supérieure, dans lequel elles deviennent linéairement séparables. Par exemple :
  - Noyau linéaire : pour une séparation linéaire.
  - Noyaux polynomial ou radial pour des séparations non linéaires.

#### Fonction de coût et régularisation

La SVM cherche à trouver l'hyperplan qui sépare au mieux les classes en maximisant la marge entre elles tout en minimisant les erreurs de classification. Ceci est formulé comme un problème d'optimisation qui minimise une fonction de coût associée.

Pour les SVM à **marge souple** (*soft margin*), le problème d'optimisation est défini par :

$$
\min_{\mathbf{w}, b} \ \frac{1}{2} \|\mathbf{w}\|^2 + C \sum_{i=1}^{n} \xi_i
$$

sous les contraintes :

$$
y_i (\mathbf{w}^\top \mathbf{x_i} + b) \geq 1 - \xi_i, \quad \xi_i \geq 0, \quad \forall i = 1, \dots, n
$$

où :

- $\mathbf{w}$ est le vecteur des poids (coefficients de l'hyperplan),
- $b$ est le biais,
- $y_i$ est la classe de l'exemple $i$ ($y_i = +1$ ou $y_i = -1$),
- $\mathbf{x_i}$ est le vecteur des caractéristiques de l'exemple $i$,
- $\xi_i$ sont les variables de relaxation (*slack variables*) qui permettent de pénaliser les erreurs de classification,
- $C$ est le paramètre de régularisation qui contrôle le compromis entre maximiser la marge et minimiser les erreurs de classification.

Le terme $\frac{1}{2} \|\mathbf{w}\|^2$ vise à maximiser la marge (en minimisant la norme de $\mathbf{w}$), tandis que le terme $C \sum_{i=1}^{n} \xi_i$ pénalise les erreurs de classification (les points mal classés ou à l'intérieur de la marge).

Un $C$ élevé donne une pénalité plus forte pour les erreurs de classification, ce qui conduit à une marge plus petite pour minimiser les erreurs. Un $C$ faible permet une marge plus large au prix de plus d'erreurs, favorisant une meilleure généralisation.

En Python, *Scikit-Learn* propose une implémentation des SVM avec la classe `SVC`.

#### Exemple

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
from sklearn.svm import SVC

# Instanciation du modèle SVM avec un noyau linéaire
svm = SVC(kernel='linear', probability=True)

# Entraînement du modèle
svm.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred_svm = svm.predict(X_test)
```

</div>

- Le paramètre `kernel` contrôle le type de noyau utilisé. Ici, nous avons choisi le noyau linéaire (`kernel='linear'`), mais d'autres options, comme `poly` pour un noyau polynomial ou `rbf` pour un noyau radial, sont également disponibles pour des séparations non linéaires.

Remarque : SVM permet aussi la régression.

### Comparaison entre régression logistique et k-NN

|    **Modèle**         |                  **Avantages**                      |         **Inconvénients**            |
|:---------------------:|-----------------------------------------------------|--------------------------------------|
| Régression logistique | Facile à interpréter, adaptée aux données linéaires | Limitée aux séparations linéaires    |
|      k-NN             | Simple, efficace pour relations locales             | Sensible aux données bruitées        |
|       SVM             | Performant pour les séparations non linéaires       | Complexe et coûteux pour grands jeux |

---

## Évaluation des modèles

### Métriques pour la régression

Pour la régression, nous utilisons l'erreur quadratique moyenne MSE ou encore sa
racine carrée (RMSE) ainsi que le coefficient de détermination $R^2$.

### Métriques pour la classification

#### 1. **Matrice de confusion**

La **matrice de confusion** est un tableau qui permet d'évaluer la performance
d'un modèle de classification en comparant les prédictions aux résultats réels.
Elle contient quatre types de prédictions :

- **Vrais positifs (VP)** : instances positives correctement classées.
- **Faux positifs (FP)** : instances négatives incorrectement classées comme positives.
- **Faux négatifs (FN)** : instances positives incorrectement classées comme négatives.
- **Vrai négatifs (VN)** : instances négatives correctement classées.

Exemple de matrice de confusion pour une classification binaire :

|                              | **Prédiction : positif** | **Prédiction : négatif** |
|------------------------------|:------------------------:|:------------------------:|
| **Classe réelle : positif**  |            VP            |            FN            |
| **Classe réelle : négatif**  |            FP            |            VN            |

Avec *scikit-learn*, on utilisera la fonction `confusion_matrix()` :

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)
```

</div>

La matrice de confusion nous permet de définir plusieurs métriques pour évaluer la performance d'un modèle de classification : la précision, le rappel et la F-mesure.

#### 2. **Précision**

La précision correspond à proportion des prédictions positives correctes parmi les prédictions positives. Elle est définie par :

$$
\text{Précision} = \frac{VP}{VP + FP}
$$

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">
  
```python
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred)
print(f'Précision : {precision}')
```

</div>

#### **Rappel**

Le rappel renseigne sur la proportion des vrais positifs parmi toutes les instances réellement positives. Il est défini par :

$$
\text{Rappel} = \frac{VP}{VP + FN}
$$

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
from sklearn.metrics import recall_score

rappel = recall_score(y_test, y_pred)
print(f'Rappel : {rappel}')
```

</div>

#### 3. **F-mesure**

La F-mesure (ou score F1) correspond à la moyenne harmonique de la précision et
du rappel. C'est une mesure globale de la performance qui est utile lorsque les
classes sont déséquilibrées. La F-mesure est définie par :

$$
F1 = 2 \times \frac{\text{Précision} \times \text{Rappel}}{\text{Précision} + \text{Rappel}}
$$

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">
  
```python
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred)
print(f'F-mesure : {f1}')
```

</div>

Remarque : Pour un ensemble de $n$ valeurs $x_1, x_2, \dots, x_n$, la moyenne harmonique est donnée par :

$$
H = \frac{n}{\sum_{i=1}^n \frac{1}{x_i}}
$$

#### 4. Rapport de classification

Il est possible de récupérer un rapport complet de ces différents métriques
en appelant la fonction `classification_report()` :

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">
  
```python
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))
```

</div>

#### 5. Exactitude

On peut aussi mesurer la performance d'un modèle de classification en calculant
son **exactitude**. L'exactitude (*accuracy*) représente la proportion
d'observations correctement classées parmi l'ensemble des observations. Elle est
définie par :

$$
\text{Exactitude} = \frac{VP + VN}{VP + VN + FP + FN}
$$

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">
  
```python
from sklearn.metrics import accuracy_score

exactitude = accuracy_score(y_test, y_pred)
print(f'Exactitude : {exactitude}')
```

</div>

#### 6. Courbe ROC et AUC

La courbe **ROC** (*receiver operating characteristic*) est un graphique qui
illustre la performance d'un modèle de classification binaire en fonction de
son taux de vrais positifs (le rappel) et de son taux de faux positifs (FPR).
Ce **FPR** (*false positive rate*) est la proportion de faux positifs parmi
les négatifs. La figure ci-dessous présente les courbes ROC de différents 
types de modèles.

![Modèles de classificateurs et courbes ROC](figures/courbes_ROC.svg)

La courbe ROC est tracée en faisant varier le seuil de décision du modèle. Elle permet
d'observer comment la capacité de classification change en fonction du seuil et de
choisir le seuil le mieux adapté.

L'aire sous la courbe ROC, ou **AUC** (*area under curve*), quantifie la qualité
du modèle. Un AUC de 1 signifie que la classification est parfaite, tandis qu'un
AUC de 0,5 représente une classification aléatoire.

##### Exemple

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">
  
```python
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Calcul de la probabilité de la classe positive
y_proba = log_reg.predict_proba(X_test)[:, 1]

# Calcul de la courbe ROC
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Affichage de la courbe ROC
plt.plot(fpr, tpr, label='Courbe ROC')
plt.plot([0, 1], [0, 1], 'k--', label='Aléatoire')
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Rappel')
plt.title('Courbe ROC')
plt.legend()
plt.show()

# Calcul de l'AUC
auc = roc_auc_score(y_test, y_proba)
print(f"Aire sous la courbe ROC (AUC) : {auc}")
```

</div>

### Validation croisée

La **validation croisée** consiste à diviser les données en plusieurs sous-ensembles
(appelés "folds"). Le modèle est entraîné sur plusieurs combinaisons de ces
sous-ensembles et testé sur les sous-ensembles restants. Cela permet d'améliorer la
**robustesse** et de réduire les risques de **sur-apprentissage** (*overfitting*).

Dans une validation croisée à $k$ sous-ensembles, chaque sous-ensemble sert de jeu de
test une fois, tandis que les $k-1$ sous-ensembles restants forment le jeu d'entraînement.
La validation croisée est mise en oeuvre avec la fonction `cross_val_score()`.

#### Exemple

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
from sklearn.model_selection import cross_val_score

# Validation croisée avec 5 sous-ensembles pour un modèle de régression logistique
scores = cross_val_score(log_reg, X, y, cv=5)
print("Scores de validation croisée (5 sous-ensembles) :", scores)
print("Précision moyenne :", scores.mean())
```

</div>

### Comparaison avec les k-plus proches voisins (k-NN)

L'algorithme des **k plus proches voisins (k-NN)** est un algorithme de classification
où la classe d'une observation est déterminée par les $k$ observations (ou voisins) les
plus proches dans le jeu de données.
Cet algorithme est souvent utilisé pour sa simplicité et permet une comparaison
intéressante avec la régression logistique.



---

## Exercices

### Exercice 1 : Analyse et visualisation des données

1. Importez les bibliothèques nécessaires

   ```python
   import pandas as pd
   import numpy as np
   import matplotlib.pyplot as plt
   import seaborn as sns
   ```

2. Chargez les données du fichier "`sonar.csv`".
3. Examinez les premières lignes du jeu de données et les statistiques descriptives pour identifier la distribution des valeurs dans chaque colonne.
4. Visualisez les valeurs des 60 caractéristiques pour les classes "R" (roche) et "M" (mine) à l'aide de cartes thermiques (*heatmaps*) et de nuages de points. Recherchez des motifs ou des différences entre les classes.

#### Solution


In [ ]:
# Votre code ici


---

### Exercice 2 : Entraînement d'un modèle de régression logistique

1. Divisez les données en un ensemble d'entraînement et un ensemble de test en utilisant la
   fonction `train_test_split()` de *Scikit-learn* (70% - 30%). Vous penserez à encoder
   "R" (roche) en 0 et "M" (mine) en 1 à l'aide de la méthode `map()` :

   ```python
   df['class'] = df['class'].map({'R': 0, 'M': 1})
   ```

2. Instanciez la classe `LogisticRegression` de manière à créer un modèle de régression
   logistique. Entraînez ce modèle à l'aide de sa méthode `fit()`), puis faites des prédictions
   sur l'ensemble de test à l'aide de sa méthode `predict()`.
3. Affichez la matrice de confusion et le rapport complet de classification pour ce modèle.

#### Solution


In [ ]:
# Votre code ici

---

### Exercice 3 : Comparaison avec les modèles k-NN et SVM

1. Entraînez un modèle k-NN avec les mêmes données ($k = 5$) et un modèle SVM
   avec un noyau radial de base (`kernel='rbf'`). Comparez leurs performances
   avec celles de la régression logistique.
2. La précision et la F-mesure sont-elles plus élevées avec le k-NN, la SVM ou
   avec la régression logistique ? Justifiez.
3. Utilisez la fonction `accuracy_score()` afin d'obtenir l'exactitude des trois
   modèles. Que constatez-vous ?
4. Utilisez la fonction `predict_proba()` pour obtenir la probabilité
   d'appartenance de chaque observation à la classe positive pour les trois
   modèles.
5. Utilisez ensuite la fonction `roc_curve()` de *Scikit-learn* pour calculer
   les valeurs nécessaires aux trois courbes ROC.
6. Tracez les courbes ROC des trois modèles sur le même graphique et interprétez
   leurs formes.
7. Utilisez la fonction `roc_auc_score()` pour calculer l'aire sous la courbe
   ROC (AUC) pour les trois modèles. Quelle est l'importance d'un AUC élevé
   pour la détection ?

#### Solution


In [ ]:
# Votre code ici


---

### Exercice 4 : Validation croisée et optimisation d'hyperparamètres

1. Utilisez la validation croisée pour tester la robustesse des trois modèles.
2. Comparez la moyenne et l'écart-type des scores pour la régression logistique,
   le modèle k-NN et la SVM.
3. Testez plusieurs valeurs de $k$ pour déterminer celle qui donne les meilleurs
   résultats pour le modèle k-NN. Pour la SVM, testez différents noyaux (`'linear'`,
   `'poly'`, `'rbf'`) et valeurs du paramètre $C$ pour déterminer la combinaison
   optimale. Pourquoi est-il important d'optimiser ces hyperparamètres ?

#### Solution


In [ ]:
# Votre code ici

---

## Conclusion

Dans cette séance, vous avez :

- mis en oeuvre une classification binaire dans le cadre de la détection ;
- entraîné trois modèles de classification binaire afin de distinguer les états : une
  régression logistique, un k-NN et une SVM ;
- utilisé des métriques d'évaluation et la validation croisée afin d'évaluer la
  robustesse et la précision des trois modèles.
- optimisé les hyperparamètres pour améliorer les performances de vos modèles.

---

## Références

**[1]** T. Sejnowski and R. Gorman. "Connectionist Bench (Sonar, Mines vs. Rocks)," UCI Machine Learning Repository, 1988. Jeu de données accessible à l'adresse : [https://doi.org/10.24432/C5T01Q](https://doi.org/10.24432/C5T01Q).

---

## Annexe

### Bibliothèque *Seaborn*

[*Seaborn*](https://seaborn.pydata.org) est une bibliothèque de visualisation en
Python qui vient en surcouche de *Matplotlib*. Elle permet de créer facilement des
graphiques statistiques multivariables. Par exemple :

- des distribution de données avec les fonctions `sns.histplot()`, `sns.kdeplot()` ;
- des nuages de points : `sns.scatterplot()` ;
- des boîtes à moustaches : `sns.boxplot()`.

#### Exemple

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
import seaborn as sns
import matplotlib.pyplot as plt

# Assurez-vous que le DataFrame 'df' est déjà chargé avec les données de 'sonar.csv'
# et que les colonnes sont correctement nommées comme dans les exercices précédents.

# Histogramme de la distribution de la caractéristique 1
sns.histplot(df['Attribute1'], kde=True)
plt.title('Distribution de la caractéristique 1')
plt.xlabel('Valeurs de Attribute1')
plt.ylabel('Fréquence')
plt.show()

# Nuage de points des caractéristiques 1 et 2
sns.scatterplot(data=df, x='f1', y='f2', hue='class')
plt.title('Nuage de points des caractéristiques 1 et 2')
plt.xlabel('Attribute1')
plt.ylabel('Attribute2')
plt.legend(title='Classe')
plt.show()

# Boxplot pour comparer la caractéristique 1 selon la cible
sns.boxplot(data=df, x='class', y='f1')
plt.title('Boxplot de la caractéristique 1 selon la cible')
plt.xlabel('Classe')
plt.ylabel('Valeurs de Attribute1')
plt.show()
```

</div>